In [3]:
from pathlib import Path
import ross as rs
import numpy as np

###################################################
import plotly.io as pio
#pio.renderers.default = "notebook"
pio.renderers.default = "browser"
###################################################

# uncomment the lines below if you are having problems with plots not showing
# import plotly.io as pio
# pio.renderers.default = "notebook"

# The Main procedure of ROSS is "Building the model + Calculating the results".(모델링 + 결과 고찰하기가 핵심!)
# SI unit as default

In [ ]:
############################ 자재 저장 안되어 있을 경우만 실행 시키기 #######################

# Section 1. Material Class

# 1.1 Creating a material
# 새로운 material 을 입력하기 위해서 사용. E, G, rho 중에 2개 입력
# material안에 있는 "name"이 실제로 저장되는 이름

# from E and G_s
steel1 = rs.Material(name="Steel", rho=7810, E=211e9, G_s=81.2e9)

# material properties used for Chilbo No.2
brake_p = rs.Material(name="brake_mat", rho=7850*0.749344, E=200e9, G_s=80e9)
lowfan_p = rs.Material(name="lowfan_mat",  rho=7850*0.801282, E=200e9, G_s=80e9)
upfan_p = rs.Material(name="upfan_mat",  rho=7850*0.400442, E=200e9, G_s=80e9)     ## rho be modified
thrcollar_p = rs.Material(name="thrcollar_mat",  rho=7850*1.191540, E=200e9, G_s=80e9)
collarkey_p = rs.Material(name="collarkey_mat",  rho=7850*1.538462, E=200e9, G_s=80e9)
slipring_p = rs.Material(name="slipring_mat", rho=7850*0.654907, E=200e9, G_s=80e9) ## rho be modified
speedsensor_p = rs.Material(name="SSG_mat",  rho=7850*1.333333, E=200e9, G_s=80e9)
rimkey_p = rs.Material(name="rimkey_mat",  rho=7850, E=200e9, G_s=80e9)


# returning attributes
#print("="*36)
#print(f"Young's Modulus: {brake_p.E}")
#print(f"Shear Modulus:    {brake_p.G_s}")

# Saving materials
brake_p.save_material()
lowfan_p.save_material()
upfan_p.save_material()
thrcollar_p.save_material()
collarkey_p.save_material()
slipring_p.save_material()
speedsensor_p.save_material()
rimkey_p.save_material()

# 사용 가능한 material properties 저장 파일 위치 확인하기(2024.12.16)
print(rs.__file__)
print("="*36)
print(rs.Material.available_materials())
#####################################################################################################################

In [4]:
# 사용 가능한 material 불러오기
steel_pp = rs.Material.load_material('Steel')
brake_pp = rs.Material.load_material('brake_mat')
lowfan_pp = rs.Material.load_material('lowfan_mat')
upfan_pp = rs.Material.load_material('upfan_mat')
thrcollar_pp = rs.Material.load_material('thrcollar_mat')
collarkey_pp = rs.Material.load_material('collarkey_mat')
slipring_pp = rs.Material.load_material('slipring_mat')
SSG_pp = rs.Material.load_material('SSG_mat')
rimkey_pp = rs.Material.load_material('rimkey_mat')


In [ ]:
############# 연산 생략 ##########
############# density, ite를 구하기 위해서 1차적으로 계산한 값

brake_disk = rs.DiskElement.from_geometry(
    n=32,   # plot 보고 수정한 station #
    material=brake_pp,
    width=200*0.001,
    i_d=1600*0.001,
    o_d=1730*0.001,
    tag="brake_disk"   
)
print(brake_disk) # Ith를 다 맞춰야됨       210/2을 이동한 값으로 적용 예정
print("="*36)
lowfan_disk = rs.DiskElement.from_geometry(
    n=32,
    material=lowfan_pp,
    width=56*0.001,
    i_d=1613*0.001,
    o_d=2100*0.001,
    tag="lowfan_disk"    
)
print(lowfan_disk) # Ith를 다 맞춰야됨      108를 이동한 값으로 사용 필요
print("="*36)
upfan_disk = rs.DiskElement.from_geometry(
    n=40,
    material=upfan_pp,
    width=56*0.001,
    i_d=1613*0.001,
    o_d=2100*0.001,
    tag="upfan_disk"    
)
print(upfan_disk) # Ith를 다 맞춰야됨    99를 이동한 값으로 사용 필요
print("="*36)
thrcollar_disk = rs.DiskElement.from_geometry(
    n=49-1,
    material=thrcollar_pp,
    width=824*0.001,
    i_d=450*0.001,
    o_d=730*0.001,
    tag="thrcollar_disk"    
)
print(thrcollar_disk) # Ith를 다 맞춰야됨       258 이동한 값으로 사용 필요
#기본적으로 density 수정해서 질량을 맞춰야됨
print("="*36)

In [5]:
#### Disk list 만들기

# 이렇게 station number를 직접 적을때는 station number가 0부터 시작했을 때 규칙을 기반으로 적기 #


# common properites
gr = 9.8 # unit: m2/s


### effective transverse inertia가 필요한 5개 disks
# runner on station 0
# 직접 inertia properties를 이용한 디스크 만들기

# runner on station 0
runner = 21364      # unit: N
runner_l = 0.3947   # unit: m
runner_m = 2200    #unit: kg
runner_Ip = 24427/(4*gr)    #unit : kg*m**2
runner_It = 0.5*runner_Ip + (runner_m*(runner_l)**2)/12    #unit : kg*m**2
runner_Ite = runner_It + runner_m*(runner_l/2)**2

print(f"runner_It on c.g.:{runner_It}")

# 직접 inertia properties를 이용한 디스크 만들기
runner_disk = rs.DiskElement(
    n=0,            # node to input must be inserted.(or an error occurs!)
    m=runner_m,
    Ip=runner_Ip,
    Id=runner_Ite,
    tag="runner_disk"  # A tag to name the element Default is None
)
print(runner_disk)
print("="*36)

watermass_disk = rs.DiskElement(
    n=0,            # node to input must be inserted.(or an error occurs!)
    m=runner_m*0.7,
    Ip=0,
    Id=0,
    tag="watermass_disk"  # A tag to name the element Default is None
)
print(watermass_disk)
print("="*36)



# brake on station 32
brake_l = 210*0.001   # unit: m  
brake_m = 400    #unit: kg
brake_Ip = 277.64    #unit : kg*m**2
brake_It = 140.16    #unit : kg*m**2
brake_d = 105*0.001 # distance between the axis of c.g and the new axis // unit: m
brake_Ite = brake_It + brake_m*(brake_d)**2
print(f"brake_It on c.g.:{brake_Ite}")

# 직접 inertia properties를 이용한 디스크 만들기
brake_disk = rs.DiskElement(
    n=32,            # node to input must be inserted.(or an error occurs!)
    m=brake_m,
    Ip=brake_Ip,
    Id=brake_Ite,
    tag="brake_disk"  # A tag to name the element Default is None
)
print(brake_disk)
print("="*36)



# lowfan on station 32
lowfan_l = 56*0.001   # unit: m  
lowfan_m = 500    #unit: kg
lowfan_Ip = 438.45    #unit : kg*m**2
lowfan_It = 219.36    #unit : kg*m**2
lowfan_d = 108*0.001 # distance between the axis of c.g and the new axis // unit: m
lowfan_Ite = lowfan_It + lowfan_m*(lowfan_d)**2
print(f"lowfan_It on c.g.:{lowfan_Ite}")

# 직접 inertia properties를 이용한 디스크 만들기
lowfan_disk = rs.DiskElement(
    n=32,            # node to input must be inserted.(or an error occurs!)
    m=lowfan_m,
    Ip=lowfan_Ip,
    Id=lowfan_Ite,
    tag="lowfan_disk"  # A tag to name the element Default is None
)
print(lowfan_disk)
print("="*36)



# upfan on station 40
upfan_l = 56*0.001   # unit: m  
upfan_m = 250    #unit: kg
upfan_Ip = 219.23    #unit : kg*m**2
upfan_It = 109.68    #unit : kg*m**2
upfan_d = 99*0.001 # distance between the axis of c.g and the new axis // unit: m
upfan_Ite = upfan_It + upfan_m*(upfan_d)**2
print(f"upfan_It on c.g.:{upfan_Ite}")

# 직접 inertia properties를 이용한 디스크 만들기
upfan_disk = rs.DiskElement(
    n=40,            # node to input must be inserted.(or an error occurs!)
    m=upfan_m,
    Ip=upfan_Ip,
    Id=upfan_Ite,
    tag="upfan_disk"  # A tag to name the element Default is None
)
print(upfan_disk)
print("="*36)



# thrcollar on station 48
thrcollar_l = 824*0.001   # unit: m  
thrcollar_m = 2000    #unit: kg
thrcollar_Ip = 188.66    #unit : kg*m**2
thrcollar_It = 207.57    #unit : kg*m**2
thrcollar_d = 258*0.001 # distance between the axis of c.g and the new axis // unit: m
thrcollar_Ite = thrcollar_It + thrcollar_m*(thrcollar_d)**2
print(f"thrcollar_It on c.g.:{thrcollar_Ite}")

# 직접 inertia properties를 이용한 디스크 만들기
thrcollar_disk = rs.DiskElement(
    n=48,            # node to input must be inserted.(or an error occurs!)
    m=thrcollar_m,
    Ip=thrcollar_Ip,
    Id=thrcollar_Ite,
    tag="thrcollar_disk"  # A tag to name the element Default is None
)
print(thrcollar_disk)
print("="*36)


runner_It on c.g.:340.1300273843537
Tag:                      runner_disk
Node:                     0
Mass           (kg):      2200.0
Diam. inertia  (kg*m**2): 425.81
Polar. inertia (kg*m**2): 623.14
brake_It on c.g.:144.57
Tag:                      brake_disk
Node:                     32
Mass           (kg):      400.0
Diam. inertia  (kg*m**2): 144.57
Polar. inertia (kg*m**2): 277.64
lowfan_It on c.g.:225.192
Tag:                      lowfan_disk
Node:                     32
Mass           (kg):      500.0
Diam. inertia  (kg*m**2): 225.19
Polar. inertia (kg*m**2): 438.45
upfan_It on c.g.:112.13025
Tag:                      upfan_disk
Node:                     40
Mass           (kg):      250.0
Diam. inertia  (kg*m**2): 112.13
Polar. inertia (kg*m**2): 219.23
thrcollar_It on c.g.:340.698
Tag:                      thrcollar_disk
Node:                     48
Mass           (kg):      2000.0
Diam. inertia  (kg*m**2): 340.7
Polar. inertia (kg*m**2): 188.66


In [6]:
# effective It가 필요없는, station 위치에 있는 disks

collarkey_disk = rs.DiskElement.from_geometry(
    n=51-1,
    material=collarkey_pp,
    width=50*0.001,
    i_d=388*0.001,
    o_d=603*0.001,     # done the mass, geometries setup 
    tag="collarkey_disk"
)
print(collarkey_disk)
print("="*36)
slipring_disk1 = rs.DiskElement.from_geometry(
    n=56-1,
    material=slipring_pp,
    width=30.5*0.001,
    i_d=132*0.001,
    o_d=590*0.001,   
    tag="slipring_disk1" 
)
print(slipring_disk1)
print("="*36)
slipring_disk2 = rs.DiskElement.from_geometry(
    n=57-1,
    material=slipring_pp,
    width=39.5*0.001,
    i_d=132*0.001,
    o_d=600*0.001, 
    tag="slipring_disk2"   
)
print(slipring_disk2)
print("="*36)
slipring_disk3 = rs.DiskElement.from_geometry(
    n=58-1,
    material=slipring_pp,
    width=39.5*0.001,
    i_d=132*0.001,
    o_d=600*0.001,   
    tag="slipring_disk3" 
)
print(slipring_disk3)
print("="*36)
SSG_disk = rs.DiskElement.from_geometry(
    n=59,
    material=SSG_pp,    
    width=29.3*0.001,
    i_d=130*0.001,
    o_d=322*0.001,  
    tag="SSG_disk"
)
print(SSG_disk)
print("="*36)
print("="*36)

# the list of total disks
list_of_disks = [runner_disk, brake_disk, lowfan_disk, upfan_disk, thrcollar_disk, collarkey_disk, slipring_disk1, slipring_disk2, slipring_disk3, SSG_disk]
print(list_of_disks)

Tag:                      collarkey_disk
Node:                     50
Mass           (kg):      101.05
Diam. inertia  (kg*m**2): 3.2682
Polar. inertia (kg*m**2): 6.4943
Tag:                      slipring_disk1
Node:                     55
Mass           (kg):      40.723
Diam. inertia  (kg*m**2): 0.93349
Polar. inertia (kg*m**2): 1.8607
Tag:                      slipring_disk2
Node:                     56
Mass           (kg):      54.638
Diam. inertia  (kg*m**2): 1.296
Polar. inertia (kg*m**2): 2.5777
Tag:                      slipring_disk3
Node:                     57
Mass           (kg):      54.638
Diam. inertia  (kg*m**2): 1.296
Polar. inertia (kg*m**2): 2.5777
Tag:                      SSG_disk
Node:                     59
Mass           (kg):      20.903
Diam. inertia  (kg*m**2): 0.15903
Polar. inertia (kg*m**2): 0.31507
[DiskElement(Id=425.81, Ip=623.14, m=2200.0, color='Firebrick', n=0, scale_factor=1.0, tag='runner_disk'), DiskElement(Id=144.57, Ip=277.64, m=400.0, color='Fir

In [7]:
### shaft의 lamination하는 경우 어떻게 element 입력하는 지 확인함

## Tips on shaft element building via Excel
# 1. Element number는 반드시 1부터 시작해야 된다.(0으로 시작하면 인식 못함) 따라서, Disk 등 added mass 넣을 때 번호 주의
# 2. Layer를 입힐 수 있음. Layer는 대부분 stiffness가 0이므로, rho만 넣어 주면 된다.
# 3. 그러나 Layer중에 stiffness가 약간 있는 경우는(실험 등에서 구한 경우) 기존 modulus에서 엄청  %를  줄여서 input으로 넣는다.(위의 material #2 경우)


## load the shaft from an excel sheet
shaft_file = Path("shaft_si7.xls")

shaft = rs.ShaftElement.from_table(
    file=shaft_file, sheet_type="Model", sheet_name="Model"
)

#shaft
# print(shaft[0])       # 1개 shaft[0] element의 properties 표시
#shaft[0].M().shape

#file_path = Path("shaft_si6.xls")
#list_of_disks = rs.DiskElement.from_table(file=file_path, sheet_name="More")



## rotor generation with shaft only 
rotor1 = rs.Rotor(shaft)

## rotor generation with shaft and disks only
rotor2 = rs.Rotor(shaft, list_of_disks)


## node 표시를 5개 마다 표시
node_increment = 5

shaft_only = rotor1.plot_rotor(nodes=node_increment)
shaft_only.show()
shaft_disk = rotor2.plot_rotor(nodes=node_increment)
shaft_disk.show()

# default는 node_increment = 1
# rotor1.plot_rotor()

In [ ]:
print(shaft[0])

In [ ]:
#### (실행시키지 말것!!!) rotor 예제이므로 생략

shaft_file = Path("shaft_si.xls")
shaft = rs.ShaftElement.from_table(
    file=shaft_file, sheet_type="Model", sheet_name="Model"
)

file_path = Path("shaft_si.xls")
list_of_disks = rs.DiskElement.from_table(file=file_path, sheet_name="More")

bearing1 = rs.BearingElement.from_table(n=7, file="bearing_seal_si.xls")
bearing2 = rs.BearingElement.from_table(n=48, file="bearing_seal_si.xls")

bearings = [bearing1, bearing2]

rotor3 = rs.Rotor(shaft, list_of_disks, bearings)

rotor3.plot_rotor()

In [12]:
### brg station number는 station 0부터 카운팅 해서 들어가야됨(즉, excel for shaft에서 -1을 해야됨)


# sheet 3개
tbnbrg = rs.BearingElement.from_table(n=4, file="bearing_seal3_si.xls", sheet_name="tbnbrg")
lowbrg = rs.BearingElement.from_table(n=27, file="bearing_seal3_si.xls", sheet_name="lowbrg")
uppbrg = rs.BearingElement.from_table(n=46, file="bearing_seal3_si.xls", sheet_name="uppbrg")

bearings = [tbnbrg, lowbrg, uppbrg]

## rotor generation with shaft and disks only
rotor3 = rs.Rotor(shaft, list_of_disks, bearings)


## node 표시를 5개 마다 표시
node_increment = 5

shaft_disk_brg = rotor3.plot_rotor(nodes=node_increment)
shaft_disk_brg.show()



# single bearing element
#file_path = Path("bearing_seal_si.xls")
#bearing = rs.BearingElement.from_table(n=0, file=file_path)
#bearing


# list of bearing elements

# nodes = list with the bearing elements nodes number
# file_path = Path("bearing_seal_si.xls")
# bearings = [rs.BearingElement.from_table(n, file_path, sheet_name=i) for i, n in enumerate(nodes)]

In [9]:
#### Bearing coefficent plotting by python(DEC/18th/2024)
## Stiffness vs frequency(hz)


# 첫 번째 플롯 (TBN Bearing)
tbnbrgfig = tbnbrg.plot(['kxx', 'kyy'], frequency_units="RPM")
tbnbrgfig.update_layout(
    title=dict(
        text="TBN Bearing",
        x=0.5,  # 타이틀 가운데 정렬
        xanchor="center"
    ),
    xaxis_title="Frequency (Hz)",
    yaxis_title="Stiffness (N/m)",
    legend=dict(x=0, y=1, xanchor="left", yanchor="top"),  # 범례 좌측 상단
)
tbnbrgfig.show()

# 두 번째 플롯 (Low Bearing)
lowbrgfig = lowbrg.plot(['kxx', 'kyy'], frequency_units="RPM")
lowbrgfig.update_layout(
    title=dict(
        text="Low Bearing",
        x=0.5,  # 타이틀 가운데 정렬
        xanchor="center"
    ),
    xaxis_title="Frequency (Hz)",
    yaxis_title="Stiffness (N/m)",
    legend=dict(x=0, y=1, xanchor="left", yanchor="top"),  # 범례 좌측 상단
)
lowbrgfig.show()

# 세 번째 플롯 (Upper Bearing)
uppbrgfig = uppbrg.plot(['kxx', 'kyy'], frequency_units="RPM")
uppbrgfig.update_layout(
    title=dict(
        text="Upper Bearing",
        x=0.5,  # 타이틀 가운데 정렬
        xanchor="center"
    ),
    xaxis_title="Frequency (Hz)",
    yaxis_title="Stiffness (N/m)",
    legend=dict(x=0, y=1, xanchor="left", yanchor="top"),  # 범례 좌측 상단
)
uppbrgfig.show()


In [13]:
#### Bearing coefficent plotting by python(DEC/18th/2024)
## Damping vs frequency(hz)


# 첫 번째 플롯 (TBN Bearing)
tbnbrgDampfig = tbnbrg.plot(['cxx', 'cyy'], frequency_units="RPM")
tbnbrgDampfig.update_layout(
    title=dict(
        text="TBN Bearing",
        x=0.5,  # 타이틀 가운데 정렬
        xanchor="center"
    ),
    xaxis_title="Frequency (Hz)",
    yaxis_title="Damping (N-s/m)",
    legend=dict(x=0, y=1, xanchor="left", yanchor="top"),  # 범례 좌측 상단
)
tbnbrgDampfig.show()

# 두 번째 플롯 (Low Bearing)
lowbrgDampfig = lowbrg.plot(['cxx', 'cyy'], frequency_units="RPM")
lowbrgDampfig.update_layout(
    title=dict(
        text="Low Bearing",
        x=0.5,  # 타이틀 가운데 정렬
        xanchor="center"
    ),
    xaxis_title="Frequency (Hz)",
    yaxis_title="Damping (N-s/m)",
    legend=dict(x=0, y=1, xanchor="left", yanchor="top"),  # 범례 좌측 상단
)
lowbrgDampfig.show()

# 세 번째 플롯 (Upper Bearing)
uppbrgDampfig = uppbrg.plot(['cxx', 'cyy'], frequency_units="RPM")
uppbrgDampfig.update_layout(
    title=dict(
        text="Upper Bearing",
        x=0.5,  # 타이틀 가운데 정렬
        xanchor="center"
    ),
    xaxis_title="Frequency (Hz)",
    yaxis_title="Damping (N-s/m)",
    legend=dict(x=0, y=1, xanchor="left", yanchor="top"),  # 범례 좌측 상단
)
uppbrgDampfig.show()


In [27]:
static = rotor3.run_static()
static.plot_deformation()


static.plot_free_body_diagram()
#static.plot_shearing_force()
#static.plot_bending_moment()

In [24]:
#### rotor model과 static_deformation이 겹치게끔 ploting


import plotly.graph_objects as go

# Rotor 객체 생성
rotor3 = rs.Rotor(shaft, list_of_disks, bearings)

# shaft_disk_brg 데이터 (노드 표시)
node_increment = 5
shaft_disk_brg = rotor3.plot_rotor(nodes=node_increment)

# static.plot_deformation 데이터
static = rotor3.run_static()
deformation_fig = static.plot_deformation()

# Plotly 그래프 초기화
fig = go.Figure()

# shaft_disk_brg 그래프 데이터 추가 (왼쪽 y축)
for trace in shaft_disk_brg['data']:
    fig.add_trace(
        go.Scatter(
            x=trace['x'],
            y=trace['y'],
            mode=trace['mode'],
            name="Shaft Disk/Bearing",
            line=trace['line']
        )
    )

# static.plot_deformation 그래프 데이터 추가 (오른쪽 y축)
for trace in deformation_fig['data']:
    fig.add_trace(
        go.Scatter(
            x=trace['x'],
            y=trace['y'],
            mode=trace['mode'],
            name="Deformation",
            line=trace['line'],
            yaxis="y2"  # 오른쪽 y축에 추가
        )
    )

# Layout 설정 (범례 숨기기)
fig.update_layout(
    title="Shaft, Disk, Bearings and Static Deformation",
    title_x=0.5,  # 타이틀 가운데 정렬
    xaxis=dict(title="Length along Shaft (m)"),
    yaxis=dict(
        title="Shaft/Disk/Bearing Data",
        side="left",
        showgrid=False
    ),
    yaxis2=dict(
        title="Static Deformation",
        side="right",
        overlaying="y",  # 왼쪽 y축과 겹치게 설정
        showgrid=False
    ),
    showlegend=False,  # 범례 숨기기
    width=1000,
    height=600
)

# 그래프 출력
fig.show()


In [ ]:
#help(enumerate)
A = [(123), (526)]
print(A)
print("="*36)
for i, j in enumerate(A):
    print(i,j)